In [1]:
import os
os.chdir("../")

# Feasibility Analysis

This notebook deals with feasibility questions which arise in asteroid mining.

In [2]:
import pandas as pd
import numpy as np

# Plotting
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objs as go

# Statistics
from scipy import stats
from statsmodels.stats.proportion import test_proportions_2indep
import statsmodels.api as sm 
from statsmodels.formula.api import ols 
from scikit_posthocs import posthoc_scheffe, posthoc_dunn

from itertools import combinations

In [3]:
df = pd.read_csv("data/Asteroid_Cleaned.csv", index_col=0)
df.shape

(1340599, 28)

## Orbital Period Consideration

In the early stages of mining, we might not have the right tech or fuel to bring asteroids near Earth for efficient mining. So, we rely on the time it takes for an asteroid to orbit. If an asteroid orbits faster (has a lower period), we can mine it more often. At this stage, we're only able to mine NEO asteroids. Now, we're wondering how the orbit time varies among different types of NEO asteroids and how it connects to what they're made of.

### Which NEO type has the shortest period?

In [4]:
data = df.loc[df.neo == 1, ["per_y", "neo_type", "composition"]].copy(deep=True)
data.shape

(33950, 3)

I'll begin by removing the outliers for each group.

In [5]:
for neo_type in data.neo_type.unique():
    for comp_type in data.composition.unique():
        mask = (data.neo_type == neo_type) & (data.composition == comp_type)

        lower_fence, upper_fence = calculate_fences(data[mask].per_y)
        outliers = (
            data[mask]
            .per_y.map(lambda x: x < lower_fence or x > upper_fence)
            .dropna()
        )
        outlier_count = outliers.sum()
        print(
            f'NEO type "{neo_type}" and {comp_type} composition had {outlier_count} outliers'
        )

        data.drop(index=outliers[outliers].index, inplace=True)

data.shape

NameError: name 'calculate_fences' is not defined

Let's look at the **period** for different NEO asteroid types.

In [ ]:
fig = make_subplots(rows=2, cols=2)
histogram_trace(1, 1, "neo_type", "Apollo", "per_y")
histogram_trace(1, 2, "neo_type", "Atira", "per_y")
histogram_trace(2, 1, "neo_type", "Aten", "per_y")
histogram_trace(2, 2, "neo_type", "Amor", "per_y")
fig.update_layout(
    height=600,
    width=800,
    title_x=0.5,
    title_text=f"Histogram<br><sup>Distribution of period for NEO asteroid types</sup>",
    legend=dict(orientation="h", yanchor="top", xanchor="center", y=-0.1, x=0.5),
)
fig.show()

Visually, all the NEO types appear to be normally distributed with a single peak and the mean centered around roughly 4 years.

In [ ]:
fig = px.box(data, y="per_y", color="neo_type")
fig.update_layout(
    height=600,
    width=800,
    title_x=0.5,
    title_text=f"Box Plot<br><sup>Distribution of period for NEO asteroid types</sup>",
    legend=dict(orientation="h", yanchor="top", xanchor="center", y=-0.1, x=0.5),
    yaxis_title="Period (Years)",
)
fig.show()

The box plot confirms that the center of the distribution, median in this case, is a little higher than 4 years. However, they have different IQRs.

To find an order among the NEO types, in terms of period, I'll perform multiple welch t-tests where my hypothesis are as follows:

**Hypothesis**

* $\mathbf{H_0}: n_1 \ge n_2$ meaning NEO type 1 has a higher or equal period than NEO type 2.

* $\mathbf{H_1}: n_1 < n_2$ meaning NEO type 1 has a lower period than NEO type 2.

**Confidence Level**

I want to perform my hypothesis tests at $99.99\%$ confidence level.

In [ ]:
for n1, n2 in combinations(data["neo_type"].unique(), 2):
    p_value = stats.ttest_ind(
        a=data[data["neo_type"] == n1].per_y,
        b=data[data["neo_type"] == n2].per_y,
        alternative="less",
        random_state=29,
        equal_var=False,
    ).pvalue

    d = decision(alpha, p_value)
    print(f"{n1} >= {n2}: {d}")

Apollo >= Atira: Fail to reject null hypothesis
Apollo >= Aten: Fail to reject null hypothesis
Apollo >= Amor: Fail to reject null hypothesis
Atira >= Aten: Fail to reject null hypothesis
Atira >= Amor: Fail to reject null hypothesis
Aten >= Amor: Fail to reject null hypothesis


From the test results, an ascending order of NEO types in terms of their period is as follows:
1. Amor
2. Aten
3. Atira
4. Apollo

This means that _Apollo_ asteroids which come in close proximity to earth have the longest periods. On the other hand, _Amor_ asteroids which are furthest away have the shortest period. 

### How does composition and NEO type interact?

In this case, I want to see how period changes when composition in taken into consideration.

In [ ]:
fig = px.line(
    data.groupby(["neo_type", "composition"]).per_y.median().reset_index(),
    x="neo_type",
    y="per_y",
    color="composition",
    text="per_y",
    markers=True,
)
fig.update_layout(
    height=600,
    width=800,
    title_x=0.5,
    title_text=f"Factor Plot<br><sup>Interaction effect between composition and NEO type</sup>",
    legend=dict(orientation="h", yanchor="top", xanchor="center", y=-0.1, x=0.5),
    yaxis_title="Period (Years)",
    
)
fig.show()

From the factor plot, it's clear that _carbonaceous_ asteroids have a varying period compared to metallic asteroids. Metallic asteroids seem to have a somewhat consistent period of roughly 4.25 years, across all NEO types. On the other hand, _carbonaceous_ asteroids' periods vary from 4.5 years to 4.05 years.

To find statistically significant proof of interaction effect and main effect, I'll conduct two-way ANOVA test.

**Hypotheses**

* $\mathbf{H_0}:$ All groups have the same mean period.

* $\mathbf{H_1}:$ At least one group has different mean period.

**Confidence Level**

As before, I'll perform the hypothesis test at $99.99\%$ confidence level.

In [ ]:
model = ols(
    "per_y ~ C(neo_type) + C(composition) + C(neo_type):C(composition)", data=data
).fit()
results = sm.stats.anova_lm(model, typ=2)
results

,sum_sq,df,F,PR(>F)
C(neo_type),4.748875,3.0,1.584441,0.190799
C(composition),15.150843,1.0,15.165032,0.000099
C(neo_type):C(composition),0.374142,3.0,0.124831,0.945459
Residual,32475.587888,32506.0,NaN,NaN


In [ ]:
results[results["PR(>F)"] <= alpha]

,sum_sq,df,F,PR(>F)
C(composition),15.150843,1.0,15.165032,0.000099


From the test, the only statistically significant effect is from _composition_. _NEO type_ and the interaction between it and _composition_ isn't significant. Now the question is does metallic asteroids have on average a shorter period than carbonaceous asteroids. I'll conduct a Welch t-test to find the statistically significant answer.

**Hypothesis Test**

* $\mathbf{H_0}: \mu_{metallic} \ge \mu_{carbonaceous}$ meaning metallic asteroids have a higher or equal average period than carbonaceous asteroids.

* $\mathbf{H_1}: \mu_{metallic} < \mu_{carbonaceous}$ meaning metallic asteroids have a lower average period than carbonaceous asteroids.

**Confidence Interval**

As before, I want to perform my hypothesis test at $99.99\%$ confidence level.

In [ ]:
p_value = stats.ttest_ind(
    a=data[data.composition == "metallic"].per_y,
    b=data[data.composition == "carbonaceous"].per_y,
    alternative="less",
    random_state=29,
    equal_var=False,
).pvalue

decision(alpha, p_value)

'Reject null hypothesis'

This means that the metallic NEO asteroids have a lower period than carbonaceous asteroids. Which is good news as we want metal mining to take priority over carbonaceous asteroids.